## StreamStats API Scraper Automatic

__Description__: Tool to automatically run the [USGS StreamStats tool](https://www.usgs.gov/mission-areas/water-resources/science/streamstats-streamflow-statistics-and-spatial-analysis-tools?qt-science_center_objects=0#qt-science_center_objects) for multiple points within a catchment and return the flow frequency curves and subcatchment boundaries. For additional details, see the [StreamStats Automation Wiki](https://github.com/Dewberry/usgs-tools/wiki/StreamStats-Automation).

__Input__: A shapefile containing the latitude and longitude of points on the stream grid for the specified state (confluence and main stem locations).

__Output__: GeoJSON file containing the delinated catchment boundary and flow frequency data for each point, as well as a CSV file containing the flow frequency data.

*Authors*: sputnam@Dewberry.com & slawler@Dewberry.com

### Load libraries and Python options:

In [1]:
import os
import sys
sys.path.append('scripts')
from stream_stats import*
import geopandas as gpd
from geojson import dump

### Specify the state abbreviation and location of the shapefile: 

In [2]:
state='MD' #The state abbreviation

path=r'C:\Users\sputnam\Documents\GitHub\usgs-tools\results\Confluences.shp' #The location of the shapefile containing the lat/lon of the catchment outlets

##### Load the shapefile:

In [3]:
use_epsg='4326' #Specify a consistent coordinate reference system

gdf=gpd.read_file(path) #Read the shapefile as a geopandas dataframe

gdf=gdf.to_crs({'init': 'epsg:{0}'.format(use_epsg)})

geom=gdf.geometry #Extract the shapley geometry for the outlets in the shapefile

print(geom.head(2))

0    POINT (-77.05564251880148 38.90503840791767)
1    POINT (-77.05021992267702 38.90984194917753)
Name: geometry, dtype: object


### Run the API tool for each point:

In [4]:
pp_dic = {} #Dictionary to store the outlet flow frequency data dictionaries

watershed_poly_dic= {} #Dictionary to store the catchment polygons (catchment boundaries)

pp_fail=[] #List to store outlet locations whose flow frequency/catchment polygons were not calculated

pp_dic, watershed_poly_dic=snappoint_analysis(geom, state, status=True) #Run the snappoint function for all catchment outlets within the shapefile and for the specified state. Option: set status=False to hide print statements

-77.05564251880148 38.90503840791767
Fetched Peak Flows
-77.05021992267702 38.90984194917753
Line 28: Expecting value: line 1 column 1 (char 0
while loop: watershed_data count: 1
Fetched Peak Flows
-77.0592223982832 38.91578705701942
Fetched Peak Flows
-77.05876482805365 38.91617132713402
Fetched Peak Flows
-77.05585793099121 38.91908907891281
Fetched Peak Flows
-77.05325172330537 38.92041496102596
Fetched Peak Flows
-77.05300161120985 38.92001214350939
Line 28: Expecting value: line 1 column 1 (char 0
while loop: watershed_data count: 1
Fetched Peak Flows
-77.0486655925942 38.927896804449645
Fetched Peak Flows
-77.04446240484044 38.92852012429679
Fetched Peak Flows
-77.0521225108642 38.93325630784865
Fetched Peak Flows
-77.04727554004153 38.93541851181741
Fetched Peak Flows
-77.04800167468886 38.93716938123049
Line 28: Expecting value: line 1 column 1 (char 0
while loop: watershed_data count: 1
Fetched Peak Flows
-77.05247502806985 38.938226099352626
Fetched Peak Flows
-77.05142113776

### Construct a summary table of the flow frequency data for each outlet:

In [5]:
ffdata=ff_summary(pp_dic) #Run this function to construct the summary table for all outlet locations

ffdata.head()    

,0,1,2,3,4,5,6,7,8,9,...,107,108,109,110,111,112,113,114,115,116
RI,,,,,,,,,,,,,,,,,,,,,
1.25,1454.031484,1443.398678,115.970397,1433.499187,138.521504,69.505962,1425.254672,1419.960886,1415.984830,44.130598,...,33.636728,130.200103,76.140217,25.894898,85.788866,69.505962,72.845153,113.064182,70.322089,52.008117
1.50,1827.112035,1813.622369,134.709483,1801.413202,160.314564,81.596016,1791.416489,1784.877519,1779.965954,59.886520,...,44.317013,150.877091,89.215962,35.422328,100.273975,81.596016,85.433124,131.402587,84.183531,61.421676
2.00,2246.639874,2229.183885,166.250453,2214.395722,197.170575,101.704986,2202.722536,2194.729302,2188.725293,78.947880,...,58.015683,185.786373,111.006541,47.132234,124.477264,101.704986,106.390940,162.249061,105.632372,76.990274
5.00,3487.312860,3456.222888,259.476690,3435.078896,305.062555,162.776963,3420.225113,3408.051360,3398.906651,157.267226,...,111.298358,288.324547,176.870527,95.957616,197.175503,162.776963,169.884926,253.547212,173.884269,124.988541
10.00,4598.342401,4555.315567,337.914196,4528.351880,395.188069,215.232371,4510.697750,4494.931840,4483.088100,235.217650,...,163.103151,374.193352,233.234940,145.454975,259.090400,215.232371,224.317898,330.441403,233.550686,166.696919


###  Save the results:

##### As a CSV:

In [6]:
cpath=os.path.join(os.getcwd(),'results') #Location to save the results

if os.path.isdir(cpath)==False: #If the desired path does not exist, create it.
    os.mkdir(cpath)    
    
ffdata.to_csv(os.path.join(cpath,'flow_frequency_data.csv')) #Save the results as a csv

##### As a geojson:

In [7]:
for i in pp_dic.keys():
    watershed_poly_dic[i]['features'][0]['ffcurve']=pp_dic[i]
    
with open('results\catchment_poly.geojson', 'w') as f:
   dump(watershed_poly_dic, f)        

# END